In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import os
# display plots in this notebook
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [10]:
model_def = 'thirdparty/CAM/models/deploy_alexnetplusCAM_imagenet.prototxt'
model_weights = 'thirdparty/CAM/models/alexnetplusCAM_imagenet.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST) 

In [11]:
[(k, v[0].data.shape) for k, v in net.params.items()]

[('conv1', (96, 3, 11, 11)),
 ('conv2', (256, 48, 5, 5)),
 ('conv3', (384, 256, 3, 3)),
 ('conv4', (384, 192, 3, 3)),
 ('conv5', (384, 192, 3, 3)),
 ('conv6', (512, 192, 3, 3)),
 ('conv7', (512, 256, 3, 3)),
 ('fc9', (1000, 512))]

In [12]:
[(k, v.data.shape) for k, v in net.blobs.items()]

[('data', (10, 3, 227, 227)),
 ('conv1', (10, 96, 55, 55)),
 ('pool1', (10, 96, 27, 27)),
 ('norm1', (10, 96, 27, 27)),
 ('conv2', (10, 256, 27, 27)),
 ('pool2', (10, 256, 13, 13)),
 ('norm2', (10, 256, 13, 13)),
 ('conv3', (10, 384, 13, 13)),
 ('conv4', (10, 384, 13, 13)),
 ('conv5', (10, 384, 13, 13)),
 ('pool5', (10, 384, 11, 11)),
 ('conv6', (10, 512, 11, 11)),
 ('conv7', (10, 512, 11, 11)),
 ('pool8_global', (10, 512, 1, 1)),
 ('fc9', (10, 1000)),
 ('prob', (10, 1000))]

network surgery

In [4]:
conv1 = net.params['conv1'][0].data
conv1_stack = np.repeat(conv1, 3, axis = 1)
conv1_stack.shape

(96, 9, 11, 11)

In [5]:
model_def = 'models/alexnetplus/deploy_alexnetplusCAM_imagenet.prototxt'
model_weights = 'thirdparty/CAM/models/alexnetplusCAM_imagenet.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST)

In [6]:
net.params['conv1-stack'][0].data[...] = conv1_stack

In [7]:
[(k, v[0].data.shape) for k, v in net.params.items()]

[('conv1-stack', (96, 9, 11, 11)),
 ('conv2', (256, 48, 5, 5)),
 ('conv3', (384, 256, 3, 3)),
 ('conv4', (384, 192, 3, 3)),
 ('conv5', (384, 192, 3, 3)),
 ('conv6', (512, 192, 3, 3)),
 ('conv7', (512, 256, 3, 3))]

In [8]:
[(k, v.data.shape) for k, v in net.blobs.items()]

[('data', (10, 9, 227, 227)),
 ('conv1-stack', (10, 96, 55, 55)),
 ('pool1', (10, 96, 27, 27)),
 ('norm1', (10, 96, 27, 27)),
 ('conv2', (10, 256, 27, 27)),
 ('pool2', (10, 256, 13, 13)),
 ('norm2', (10, 256, 13, 13)),
 ('conv3', (10, 384, 13, 13)),
 ('conv4', (10, 384, 13, 13)),
 ('conv5', (10, 384, 13, 13)),
 ('pool5', (10, 384, 11, 11)),
 ('conv6', (10, 512, 11, 11)),
 ('conv7', (10, 512, 11, 11)),
 ('pool8_global', (10, 512, 1, 1))]

In [9]:
net.save('models/alexnetplus/alexnetplusCAM_imagenet.caffemodel')